In [1]:
# Use autoplot 207's code for this app
import sys
from datetime import datetime

import numpy as np
from pyproj import Transformer

import geopandas as gpd
import pandas as pd
from iemweb.autoplot.scripts200.p207 import USEME, add_zeros, compute_grid_bounds, do_analysis, load_data
from matplotlib.patches import Rectangle
from pyiem.nws.vtec import NWS_COLORS
from pyiem.plot import MapPlot, nwssnow
from pyiem.database import get_sqlalchemy_conn
from pyiem.reference import Z_OVERLAY
from pyiem.util import utc
from shapely.geometry import Point

In [11]:
STORM_NUMBER = 14
REQUIRES_DAYS = 2
WINTER = "2025-2026"
TITLE = "1 February 2026"
SUBTITLE = "8 AM 2 February 2026"
SETPOINT_LOCS = {}
# naive US Central local time
sts = datetime(2026, 2, 1, 3)
ets = datetime(2026, 2, 25, 18)
# Get available data
ctx = {
    "coop": "yes",
    "cocorahs": "yes",
    "t": "state",
    "sz": 30,
    "z": "yes",
    "f": "linear",
    "v": "snow",
    "wfo": "DMX",
}
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, "IA")
df = load_data(ctx, sts, ets)
df = df[~df["nwsli"].isin(["DSXI4", "DMX"])]
# add zeros and QC
df = add_zeros(df, ctx)
df.to_csv("raw.csv")

In [12]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
            """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
            conn,
            params=(sts - datetime.timedelta(days=1), ets),
        )
    print(df[df["state"] == "IA"])
    mp.plot_values(
        df.lon.values,
        df.lat.values,
        df.magnitude.values,
        fmt="%.2f",
        labelbuffer=1,
        color="purple",
    )


def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE,),
        subtitle=(
            f"Snowfall totals till {SUBTITLE} from NWS COOP, LSR, CoCoRaHS Reports; "
            f"IEM {WINTER} Winter Storm #{STORM_NUMBER}"
        ),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2["val"] >= lower) & (df2["val"] < upper)]
    print(df[df["state"] == "IA"])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(
            df_useme_plot["lon"],
            df_useme_plot["lat"],
            df_useme_plot["val"].values,
            "%s",
            labels=df_useme_plot["nwsli"].values,
            textsize=10,
            labeltextsize=10,
            labelbuffer=1,
        )
    return mp

In [13]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(
            SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1]
        )
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True


def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.0
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz / 2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz / 2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1

def plot_warnings(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            """
            SELECT simple_geom from warnings w JOIN ugcs u on (w.gid = u.gid) WHERE w.phenomena = 'BZ'
            and w.issue > '2026-01-21 00:00' and w.issue < '2026-01-30 00:00' and substr(w.ugc, 1, 2) = 'IA'
            and vtec_year = 2026
            """,
            conn,
            geom_col="simple_geom",
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["BZ.W"],
        facecolor="None",
        zorder=Z_OVERLAY,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["BZ.W"], fc="None")
    p1 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0, p1), ("Blizzard Warning", "Snow Squall Warning"), loc=1).set_zorder(
        1000
    )

def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(datetime(2026, 1, 21), ets),
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor='k',
        facecolor="None",
        zorder=Z_OVERLAY,
        linewidth=4,
    )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=Z_OVERLAY,
        linewidth=2,
    )
    #p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    #mp.panels[0].ax.legend((p0,), ("Snow Squall Warning",), loc=1).set_zorder(
    #    1000
    #)

In [14]:
# Optional filter for requiring_days
if REQUIRES_DAYS > 1:
    df = df[(df["source"] == "LSR") | (df["count"] >= REQUIRES_DAYS)]
print(df[df["wfo"] == "DMX"]["source"].value_counts())

source
COOP        10
COCORAHS    10
Name: count, dtype: int64


In [23]:
def main():
    setpoints = [
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = [
        'MCWI4',
        'IA-BT-16',
        'IA-JH-1',
    ]
    if cull:
        df.loc[df["nwsli"].isin(cull), USEME] = False
    hardcode = [
        ('WMTI4', 0.9),
        ('ALXI4', 1.9),
    ]
    for nwsli, val in hardcode:
        df.loc[df["nwsli"] == nwsli, "val"] = val

    ctx["csector"] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0.5, upper=1)
    #draw_setpoints(mp)

    #plot_warnings(mp)
    #plotsqw(mp)
    res = mp.postprocess(filename="260202.png")
    mp.close()


main()

     state  wfo  val        lon        lat                             geo  \
579     IA  DVN  1.1 -90.920000  41.860000  POINT (750341.807 -307556.418)   
616     IA  DVN  1.0 -91.860000  42.630000  POINT (664715.937 -230414.731)   
651     IA  OAX  0.5 -96.020000  41.820000  POINT (329778.224 -345564.417)   
677     IA  DVN  0.3 -92.010000  41.590000  POINT (663476.593 -346841.588)   
680     IA  DVN  0.3 -90.580000  41.610000   POINT (781376.95 -332047.545)   
...    ...  ...  ...        ...        ...                             ...   
2620    IA  DMX  0.0 -93.596850  40.585320  POINT (540447.262 -469804.248)   
2626    IA  DMX  0.0 -92.488506  40.776750  POINT (631839.225 -440710.083)   
2635    IA  DMX  0.0 -94.910200  40.800600  POINT (428375.407 -453631.061)   
2637    IA  DMX  0.0 -93.154303  40.681456  POINT (576846.479 -456149.939)   
2668    IA  OAX  0.0 -95.698426  40.724626  POINT (362540.206 -465841.464)   

      used_for_analysis    nwsli  plotme    source  count  xcel

## 